# e-Greedy Example

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon-system -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon-system 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


In [7]:
import json
import time

In [8]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [12]:
VERSION=!cat ../../../version.txt
VERSION=VERSION[0]
VERSION

'1.6.0-dev'

## E-Greedy Test

In [27]:
%%writetemplate egreedy.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: egreedy
spec:
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          name: classifier-1
        - image: seldonio/mock_classifier:{VERSION}
          name: classifier-2
        - image: seldonio/mock_classifier:{VERSION}
          name: classifier-3
        - image: seldonio/mab_epsilon_greedy:{VERSION}
          name: eg-router
    graph:
      children:
      - name: classifier-1
        type: MODEL
      - name: classifier-2
        type: MODEL
      - name: classifier-3
        type: MODEL
      name: eg-router
      parameters:
      - name: n_branches
        type: INT
        value: '3'
      - name: epsilon
        type: FLOAT
        value: '0.3'
      - name: verbose
        type: BOOL
        value: '1'
      type: ROUTER
    svcOrchSpec:
      env: 
      - name: SELDON_ENABLE_ROUTING_INJECTION
        value: 'true'
    name: multi-models-predictor
    replicas: 1


In [28]:
!kubectl apply -f egreedy.yaml

seldondeployment.machinelearning.seldon.io/egreedy configured


In [42]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=egreedy -o jsonpath='{.items[0].metadata.name}')

deployment "seldon-82ee46d72f9cced67f54b7d4b2389974" successfully rolled out


In [57]:
rewards = {0: 0, 1: 1, 2: 0}  # make model 1 the best

In [60]:
import requests

for i in range(0, 50):
    req = {"data": {"ndarray": [[1.0, 2.0, 5.0]]}}
    res_raw = requests.post(
        "http://localhost:8003/seldon/seldon/egreedy/api/v1.0/predictions", json=req
    )
    res = res_raw.json()
    feedback = {
        "request": req,
        "response": res,
        "reward": rewards[res["meta"]["routing"]["eg-router"]],
    }
    res_raw = requests.post(
        "http://localhost:8003/seldon/seldon/egreedy/api/v1.0/feedback", json=feedback
    )
    assert res_raw.status_code == 200

We should see the best branch is 1

In [66]:
!kubectl logs $(kubectl get pods -l seldon-deployment-id=egreedy -o jsonpath='{.items[0].metadata.name}') eg-router --tail=10

2021-01-20 11:02:11,759 - EpsilonGreedy:route:116 - INFO:  routing type: <class 'numpy.int64'>
2021-01-20 11:02:11,784 - EpsilonGreedy:send_feedback:120 - DEBUG:  Sending feedback with reward 1.0 and truth []
2021-01-20 11:02:11,785 - EpsilonGreedy:send_feedback:121 - DEBUG:  Prev success # [3, 32, 0]
2021-01-20 11:02:11,785 - EpsilonGreedy:send_feedback:122 - DEBUG:  Prev tries # [13, 38, 9]
2021-01-20 11:02:11,785 - EpsilonGreedy:send_feedback:123 - DEBUG:  Prev best branch: 1
2021-01-20 11:02:11,785 - EpsilonGreedy:send_feedback:126 - DEBUG:  n_success: 1, n_failures: 0
2021-01-20 11:02:11,785 - EpsilonGreedy:send_feedback:139 - DEBUG:  New success # [3, 33, 0]
2021-01-20 11:02:11,785 - EpsilonGreedy:send_feedback:140 - DEBUG:  New tries # [13, 39, 9]
2021-01-20 11:02:11,785 - EpsilonGreedy:send_feedback:141 - DEBUG:  Branch values [0.23076923076923078, 0.8461538461538461, 0.0]
2021-01-20 11:02:11,785 - EpsilonGreedy:send_feedback:142 - DEBUG:  New best branch: 1


Lets change the reward to return 1 for the first branch and rerun

In [67]:
rewards = {0: 1, 1: 0, 2: 0}  # make model 0 the best

In [70]:
import requests

for i in range(0, 100):
    req = {"data": {"ndarray": [[1.0, 2.0, 5.0]]}}
    res_raw = requests.post(
        "http://localhost:8003/seldon/seldon/egreedy/api/v1.0/predictions", json=req
    )
    res = res_raw.json()
    feedback = {
        "request": req,
        "response": res,
        "reward": rewards[res["meta"]["routing"]["eg-router"]],
    }
    res_raw = requests.post(
        "http://localhost:8003/seldon/seldon/egreedy/api/v1.0/feedback", json=feedback
    )
    assert res_raw.status_code == 200

Now branch 0 should be the best

In [71]:
!kubectl logs $(kubectl get pods -l seldon-deployment-id=egreedy -o jsonpath='{.items[0].metadata.name}') eg-router --tail=10

2021-01-20 11:06:22,710 - EpsilonGreedy:route:116 - INFO:  routing type: <class 'int'>
2021-01-20 11:06:22,723 - EpsilonGreedy:send_feedback:120 - DEBUG:  Sending feedback with reward 0.0 and truth []
2021-01-20 11:06:22,723 - EpsilonGreedy:send_feedback:121 - DEBUG:  Prev success # [86, 33, 0]
2021-01-20 11:06:22,723 - EpsilonGreedy:send_feedback:122 - DEBUG:  Prev tries # [96, 83, 31]
2021-01-20 11:06:22,723 - EpsilonGreedy:send_feedback:123 - DEBUG:  Prev best branch: 0
2021-01-20 11:06:22,723 - EpsilonGreedy:send_feedback:126 - DEBUG:  n_success: 0, n_failures: 1
2021-01-20 11:06:22,723 - EpsilonGreedy:send_feedback:139 - DEBUG:  New success # [86, 33, 0]
2021-01-20 11:06:22,723 - EpsilonGreedy:send_feedback:140 - DEBUG:  New tries # [96, 84, 31]
2021-01-20 11:06:22,723 - EpsilonGreedy:send_feedback:141 - DEBUG:  Branch values [0.8958333333333334, 0.39285714285714285, 0.0]
2021-01-20 11:06:22,723 - EpsilonGreedy:send_feedback:142 - DEBUG:  New best branch: 0
